In [3]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

In [16]:
n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]

In [17]:
train_subj_docs = subj_docs[:80]
test_subj_docs = subj_docs[80:100]
train_obj_docs = obj_docs[:80]
test_obj_docs = obj_docs[80:100]

In [19]:
training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs

In [18]:
sentiment_analyzer = SentimentAnalyzer()

In [21]:
all_words_neg = sentiment_analyzer.all_words([mark_negation(doc) for doc in training_docs])

In [26]:
len(all_words_neg)

3799

In [24]:
unigram_feats = sentiment_analyzer.unigram_word_feats(all_words_neg, min_freq=4)

In [25]:
len(unigram_feats)

83

In [27]:
sentiment_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

In [28]:
training_set = sentiment_analyzer.apply_features(training_docs)
testing_set = sentiment_analyzer.apply_features(testing_docs)

In [30]:
trainer = NaiveBayesClassifier.train
clf = sentiment_analyzer.train(trainer, training_set)

Training classifier


In [33]:
sentiment_analyzer.evaluate(testing_set)

Evaluating NaiveBayesClassifier results...


{'Accuracy': 0.8,
 'F-measure [obj]': 0.8,
 'F-measure [subj]': 0.8,
 'Precision [obj]': 0.8,
 'Precision [subj]': 0.8,
 'Recall [obj]': 0.8,
 'Recall [subj]': 0.8}

In [35]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [36]:
sent = "VADER is smart, handsome, and funny."

In [37]:
sentiment = SentimentIntensityAnalyzer()

In [39]:
sentiment.polarity_scores(sent)

{'compound': 0.8316, 'neg': 0.0, 'neu': 0.254, 'pos': 0.746}

In [40]:
sentences = [
"VADER is smart, handsome, and funny.", # positive sentence example
"VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
"VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
"VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
"VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
"VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
"The book was good.",         # positive sentence
"The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
"The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
"A really bad, horrible book.",       # negative sentence with booster words
"At least it isn't a horrible book.", # negated negative sentence with contraction
":) and :D",     # emoticons handled
"",              # an empty string is correctly handled
"Today sux",     #  negative slang handled
"Today sux!",    #  negative slang with punctuation emphasis handled
"Today SUX!",    #  negative slang with capitalization emphasis
"Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
]

In [41]:
paragraph = "It was one of the worst movies I've seen, despite good reviews. \
Unbelievably bad acting!! Poor direction. VERY poor production. \
The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

In [42]:
from nltk import tokenize
lines_list = tokenize.sent_tokenize(paragraph)
sentences.extend(lines_list)

In [43]:
tricky_sentences = [
"Most automated sentiment analysis tools are shit.",
"VADER sentiment analysis is the shit.",
"Sentiment analysis has never been good.",
"Sentiment analysis with VADER has never been this good.",
"Warren Beatty has never been so entertaining.",
"I won't say that the movie is astounding and I wouldn't claim that \
the movie is too banal either.",
"I like to hate Michael Bay films, but I couldn't fault this one",
"It's one thing to watch an Uwe Boll film, but another thing entirely \
to pay for it",
"The movie was too good",
"This movie was actually neither that funny, nor super witty.",
"This movie doesn't care about cleverness, wit or any other kind of \
intelligent humor.",
"Those who find ugly meanings in beautiful things are corrupt without \
being charming.",
"There are slow and repetitive parts, BUT it has just enough spice to \
keep it interesting.",
"The script is not fantastic, but the acting is decent and the cinematography \
is EXCELLENT!",
"Roger Dodger is one of the most compelling variations on this theme.",
"Roger Dodger is one of the least compelling variations on this theme.",
"Roger Dodger is at least compelling as a variation on the theme.",
"they fall in love with the product",
"but then it breaks",
"usually around the time the 90 day warranty expires",
"the twin towers collapsed today",
"However, Mr. Carter solemnly argues, his client carried out the kidnapping \
under orders and in the ''least offensive way possible.''"
]

In [44]:
sentences.extend(tricky_sentences)

In [45]:
sid = SentimentIntensityAnalyzer()

In [47]:
for sentence in sentences:
    print (sentence)
    print (sid.polarity_scores(sentence))
    print ("\n")

VADER is smart, handsome, and funny.
{'neg': 0.0, 'pos': 0.746, 'neu': 0.254, 'compound': 0.8316}


VADER is smart, handsome, and funny!
{'neg': 0.0, 'pos': 0.752, 'neu': 0.248, 'compound': 0.8439}


VADER is very smart, handsome, and funny.
{'neg': 0.0, 'pos': 0.701, 'neu': 0.299, 'compound': 0.8545}


VADER is VERY SMART, handsome, and FUNNY.
{'neg': 0.0, 'pos': 0.754, 'neu': 0.246, 'compound': 0.9227}


VADER is VERY SMART, handsome, and FUNNY!!!
{'neg': 0.0, 'pos': 0.767, 'neu': 0.233, 'compound': 0.9342}


VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!
{'neg': 0.0, 'pos': 0.706, 'neu': 0.294, 'compound': 0.9469}


The book was good.
{'neg': 0.0, 'pos': 0.492, 'neu': 0.508, 'compound': 0.4404}


The book was kind of good.
{'neg': 0.0, 'pos': 0.343, 'neu': 0.657, 'compound': 0.3832}


The plot was good, but the characters are uncompelling and the dialog is not great.
{'neg': 0.327, 'pos': 0.094, 'neu': 0.579, 'compound': -0.7042}


A really bad, horrible book.
{'neg':